In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm

!pip install git+https://github.com/openai/CLIP.git

/bin/bash: line 1: conda: command not found
  Cloning https://github.com/openai/CLIP.g to /tmp/pip-req-build-3unq7kud
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.g /tmp/pip-req-build-3unq7kud
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/openai/CLIP.g /tmp/pip-req-build-3unq7kud did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/openai/CLIP.g /tmp/pip-req-build-3unq7kud did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [3]:
!gdown 1d4v9MsYBdrpcOmup0CND75Y-Hexmo3-t
!unzip -qq "/content/FULL [Heineken Vietnam] Developer Resources-20240603T132806Z-001.zip" -d "/content/dataset"
!rm "/content/FULL [Heineken Vietnam] Developer Resources-20240603T132806Z-001.zip"


Downloading...
From (original): https://drive.google.com/uc?id=1d4v9MsYBdrpcOmup0CND75Y-Hexmo3-t
From (redirected): https://drive.google.com/uc?id=1d4v9MsYBdrpcOmup0CND75Y-Hexmo3-t&confirm=t&uuid=5c0e5659-fdc6-4109-9a24-236d4fcaa46e
To: /content/FULL [Heineken Vietnam] Developer Resources-20240603T132806Z-001.zip
100% 1.10G/1.10G [00:25<00:00, 43.6MB/s]


In [45]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os

In [ ]:
!pip install open_clip_torch

In [41]:
import torch
import open_clip as clip
from PIL import Image
device = "cuda" if torch.cuda.is_available() else "cpu"

model, _, preprocess = clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k', device=device)
model.eval()
tokenizer = clip.get_tokenizer('ViT-B-32')
context = ["in the outdoor_venue",
           "People are shopping",
           "in the bar or the pub or the night_club",
           "in the restaurant",
           "in the store",
           "in the big supermarket",
           "A display counter"
           "A party or the celebration",
           "A gathering",
           "A happy hour, or a fun time",
           "A fun time",
           "There are some beer carton",
           "There are some Promotional Material: Signage, billboard, poster, standee ",
           "People is taking photo"
]

text = tokenizer(context).to(device)



In [46]:
results_list = []

dataset_path = "/content/dataset/FULL [Heineken Vietnam] Developer Resources"

for filename in os.listdir(dataset_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
        image_path = os.path.join(dataset_path , filename)
        raw_image = Image.open(image_path)
        image = preprocess(raw_image).unsqueeze(0).to(device)

        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            max_value, max_index = torch.max(text_probs, dim=1)
            ct = context[max_index]

            object = {
                "filename": filename,
                "context": ct,
                "text_probs": max_value
            }
            results_list.append(object)



/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [47]:
import json


In [59]:
with open('/content/drive/MyDrive/Colab Notebooks/HACKHCMC2024/context_result.json', 'w') as json_file:
    json.dump(copy, json_file, indent=4)

In [55]:
len(copy)

1484